In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import nltk #language processing
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords

#import re
#import emoji

path = '/Users/nesli/Desktop/SPRING2022/DSCC465/Kaggle_istanbul/'  

import os
os.chdir(path)


data_train = pd.read_csv(path+"congressional_tweet_training_data.csv",names=None)
data_test= pd.read_csv("congressional_tweet_test_data.csv",names=None)

import random
random.seed(465) #seed  

In [2]:
data_train['party_class']=data_train['party_id'].apply(lambda x: 0 if x=='D' else 1)

In [3]:
data_train.dropna(subset=['full_text'],inplace=True) #First, I drop the rows with NA in the text column
data_train

,favorite_count,full_text,hashtags,retweet_count,year,party_id,party_class
0,0,"b""RT @KUSINews: One of our longtime viewers wa...",KUSI,10,2017.0,R,1
1,258,"b""Today I'm urging the @CDCgov to immediately ...",Coronavirus,111,2020.0,R,1
2,0,"b'Tomorrow, #MO03 seniors graduate from Calvar...",MO03,2,2014.0,R,1
3,9,b'Congrats to #TeamUSA and Canton Native @JGre...,TeamUSA WorldJuniors,3,2017.0,R,1
4,3,b'Pleased to support @amergateways at their Ju...,ImmigrantHeritageMonth,3,2019.0,D,0
...,...,...,...,...,...,...,...
592798,3,"b'This time, it focused on careers in #publics...",publicservice publicsafety,0,2017.0,R,1
592799,5,"b'.#StormyDaniels, #MichaelWolfe, #JamesComey ...",StormyDaniels MichaelWolfe JamesComey,1,2018.0,R,1
592800,33,b'@NRDems The American people deserve the trut...,CultureOfCorruption,14,2020.0,D,0
592801,4,b'Only 2 weeks left to submit your #app to the...,app copolitics CAC16 HouseOfCode co06,3,2016.0,R,1


In [4]:
data_train=data_train.reset_index(drop=True) #to fix the indices

In [5]:
data_test['party_class']=data_test['party'].apply(lambda x: 0 if x=='D' else 1)

In [6]:
Id=data_test['Id']

In [7]:
text=data_train['hashtags'] 
text2=data_test['hashtags'] 

In [8]:
frames = [text, text2]
  
result = pd.concat(frames)
display(result)

0                           KUSI
1                    Coronavirus
2                           MO03
3           TeamUSA WorldJuniors
4         ImmigrantHeritageMonth
                   ...          
264995                ExtendCHIP
264996                 ObamaCare
264997            MissTeenUSA CT
264998                     mosen
264999           ConstitutionDay
Name: hashtags, Length: 857803, dtype: object

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf=TfidfVectorizer()
X=tfidf.fit_transform(result)

In [10]:
X_train=X[0:592803]
y_train=data_train['party_class']
X_test=X[592803:857803]
y_test=data_test['party_class']

In [11]:
def run_SVM(clf,X_train,X_test,y_train):
    clf.fit(X_train,y_train)
    y_test=clf.predict(X_test)

In [12]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [13]:
%%time
clf=LinearSVC()
run_SVM(clf,X_train,X_test,y_train)

CPU times: user 7.3 s, sys: 132 ms, total: 7.43 s
Wall time: 11.2 s


In [14]:
clf.predict(X_test)

array([1, 0, 0, ..., 0, 1, 1])

In [15]:
y_test=clf.predict(X_test)

In [16]:
clf.fit(X_train,y_train)

LinearSVC()

In [19]:
y_test=pd.DataFrame(data=y_test, columns=['party'])

In [20]:
y_test.to_csv('svm_y_test_01.csv')

In [21]:
print(y_test)

        party
0           1
1           0
2           0
3           1
4           1
...       ...
264995      0
264996      1
264997      0
264998      1
264999      1

[265000 rows x 1 columns]


In [22]:
#y_test=pd.DataFrame(data=y_test, columns=['party'])
y_test['party']=y_test['party'].apply(lambda x: 'D' if x==0 else 'R')
y_test.insert(0, "Id", Id)

In [23]:
y_test

,Id,party
0,0,R
1,1,D
2,2,D
3,3,R
4,4,R
...,...,...
264995,264995,D
264996,264996,R
264997,264997,D
264998,264998,R


In [24]:
y_test.to_csv('svm_y_test.csv')

## References

[1] https://www.youtube.com/watch?v=qvq_eLhgxDg&ab_channel=KGPTalkie